<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


In [ ]:
# pip install requests
# pip install tqdm
# pip install "transformers>=4.33.2"

# 不同BPE编码实现比较

<br>
&nbsp;

## tiktoken中的BPE

In [1]:
from importlib.metadata import version
print("tiktoken version: ", version("tiktoken"))

tiktoken version:  0.7.0


In [2]:
import tiktoken

tik_tokenizer = tiktoken.get_encoding("gpt2")
text = "Hello, world. Is this-- a test?"

In [3]:
integers = tik_tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [4]:
strings = tik_tokenizer.decode(integers)
print(strings)

Hello, world. Is this-- a test?


In [5]:
print(tik_tokenizer.n_vocab)  # ch02/02_bonus_bytepair-encoder/gpt2_model/encoder.json中的词汇数量就是50257

50257


<br>
&nbsp;

## GPT-2中原始实现的BPE

In [6]:
from bpe_openai_gpt2 import get_encoder, download_vocab

In [7]:
download_vocab()

Fetching encoder.json: 1.04Mit [00:09, 112kit/s]                                                    
Fetching vocab.bpe: 457kit [00:24, 18.6kit/s]                                                       


In [8]:
orig_tokenizer = get_encoder(model_name="gpt2_model", models_dir=".")

In [9]:
integers = orig_tokenizer.encode(text)

print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [10]:
strings = orig_tokenizer.decode(integers)

print(strings)

Hello, world. Is this-- a test?


<br>
&nbsp;

## Huggingface的transformers库中的BPE

In [11]:
import transformers
transformers.__version__

'4.41.1'

In [ ]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

from transformers import GPT2Tokenizer
hf_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
hf_tokenizer(strings)["input_ids"]

In [ ]:
from transformers import GPT2TokenizerFast

hf_tokenizer_fast = GPT2TokenizerFast.from_pretrained("gpt2")

In [ ]:
hf_tokenizer_fast(strings)["input_ids"]

<br>
&nbsp;

## 自定义从头实现的BPE

In [13]:
import os
import sys
import io
import nbformat
import types


def import_from_notebook():
    def import_definitions_from_notebook(fullname, names):
        current_dir = os.getcwd()
        path = os.path.join(current_dir, "..", "05_bpe-from-scratch", fullname + ".ipynb")
        path = os.path.normpath(path)

        # Load the notebook
        if not os.path.exists(path):
            raise FileNotFoundError(f"Notebook file not found at: {path}")

        with io.open(path, "r", encoding="utf-8") as f:
            nb = nbformat.read(f, as_version=4)

        # Create a module to store the imported functions and classes
        mod = types.ModuleType(fullname)
        sys.modules[fullname] = mod

        # Go through the notebook cells and only execute function or class definitions
        for cell in nb.cells:
            if cell.cell_type == "code":
                cell_code = cell.source
                for name in names:
                    # Check for function or class definitions
                    if f"def {name}" in cell_code or f"class {name}" in cell_code:
                        exec(cell_code, mod.__dict__)
        return mod

    fullname = "bpe-from-scratch"
    names = ["BPETokenizerSimple"]

    return import_definitions_from_notebook(fullname, names)

In [14]:
imported_module = import_from_notebook()
BPETokenizerSimple = getattr(imported_module, "BPETokenizerSimple", None)

tokenizer_gpt2 = BPETokenizerSimple()
tokenizer_gpt2.load_vocab_and_merges_from_openai(
    vocab_path=os.path.join("gpt2_model", "encoder.json"),
    bpe_merges_path=os.path.join("gpt2_model", "vocab.bpe")
)

In [15]:
integers = tokenizer_gpt2.encode(text)

print(integers)

[1544, 18798, 11, 995, 13, 1148, 256, 5303, 82, 438, 257, 1332, 30]


<br>
&nbsp;

## 一个快速的性能benchmark

In [18]:
with open("../01_main-chapter-code/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

### 原始的OpenAI GPT-2 分词器

In [19]:
%timeit orig_tokenizer.encode(raw_text)

11 ms ± 298 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Tiktoken中的OpenAI GPT-2 分词器

In [20]:
%timeit tik_tokenizer.encode(raw_text)

2.37 ms ± 120 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Hugging Face中的OpenAI GPT-2 分词器

In [ ]:
%timeit hf_tokenizer(raw_text)["input_ids"]

In [ ]:
%timeit hf_tokenizer_fast(raw_text)["input_ids"]

### 自定义的 GPT-2 分词器

In [25]:
%timeit tokenizer_gpt2.encode(raw_text)

6.76 ms ± 136 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
